In [26]:
# from newspaper import Article
# import nltk

# url = 'https://www.yahoo.com/news/labor-board-accuses-apple-suppressing-222846876.html'
# article = Article(url)

# try:
#     article.download()
#     article.parse()
#     article.nlp()  # Run NLP after parsing

#     # Print results
#     print("Summary:", article.summary)
# except Exception as e:
#     print("An error occurred:", e)

Summary: The National Labor Relations Board last week accused Apple of trying to prevent employees from discussing pay equity and pressuring an engineer who attempted to circulate a survey about wages to quit.
Apple managers allegedly threatened other employees who posted on social media and in Slack and spoke to the media about workplace concerns, according to the complaint.
I don’t regret standing up to Apple and I will continue to fight for the rights of laborers," Scarlett said.
The NLRB is also looking to force Apple to reinstate Scarlett, compensate her for lost pay and issue an apology letter.
The complaints by federal regulators highlight ongoing turmoil around organizing efforts by Apple employees both at the iPhone maker's corporate headquarters and at retail stores.


In [27]:
from newspaper import Article
import newspaper
import pandas as pd
from langdetect import detect

In [ ]:
def scrape_tesla_news(url):
    """
    Scrapes Tesla news from the given Yahoo Finance URL.

    Args:
        url: The URL of the Yahoo Finance page for Tesla news.

    Returns:
        A pandas DataFrame with columns 'title', 'summary', and 'date'.
    """

    paper = newspaper.build(url)
    articles = paper.articles

    data = []
    n = 0
    for article in articles:
        if n > 30:
            break
        try:   
            article.download()
            article.parse()
            text = article.text
            language = detect(text)
        except:
            print('error occured')

        print(language)
        if language == "en":
            print('english')
            data.append({'title': article.title,
                        'text': text,
                        'date': article.publish_date,
                        'url': article.url})
        n += 1

    df = pd.DataFrame(data)
    return df

In [33]:
# Replace with the actual Yahoo Finance URL for Tesla news
url = "https://finance.yahoo.com/quote/AAPL/news/"

df = scrape_tesla_news(url)

# Save the DataFrame to a CSV file
# df.to_csv('tesla_news.csv', index=False)

df

fr
en
english
en
english
en
english
en
english
en
english
en
english
en
english
de


Building prefix dict from c:\Users\mamin\AppData\Local\Programs\Python\Python312\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\mamin\AppData\Local\Temp\jieba.cache
Loading model cost 0.9694051742553711 seconds.
Prefix dict has been built succesfully.


ko
zh-tw
ko
ko
ko


LangDetectException: No features in text.